# Instalación de dependencias

# Bibliotecas iniciales

In [6]:
import random # Selección de respuesta aleatoria.
import json # Corpus.
import pickle # Serialización.
import numpy as np

import nltk # Natural Language Toolkit.
from nltk.stem import WordNetLemmatizer # Lematización.


from tensorflow.keras.models import load_model

In [7]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_tienda.h5')

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [9]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [10]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    return return_list

In [11]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [16]:
print("GO! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

GO! Bot is running!
Tanto NVIDIA como AMD ofrecen soluciones para tus horas de diversion. Si buscas lo mejor del mercado, te recomendamos las siguientes tarjetas graficas: NVIDIA GeForce RTX 3060TI y AMD Radeon RX 6700XT. Si nunca quieres quedarte corto de VRAM y tu presupuesto es menor, tenemos la RTX 3060 todoterreno
Tanto NVIDIA como AMD ofrecen soluciones para tus horas de diversion. Si buscas lo mejor del mercado, te recomendamos las siguientes tarjetas graficas: NVIDIA GeForce RTX 3060TI y AMD Radeon RX 6700XT. Si nunca quieres quedarte corto de VRAM y tu presupuesto es menor, tenemos la RTX 3060 todoterreno
Tanto NVIDIA como AMD ofrecen soluciones para tus horas de diversion. Si buscas lo mejor del mercado, te recomendamos las siguientes tarjetas graficas: NVIDIA GeForce RTX 3060TI y AMD Radeon RX 6700XT. Si nunca quieres quedarte corto de VRAM y tu presupuesto es menor, tenemos la RTX 3060 todoterreno
De nada! Hasta luego!
De nada! Gracias por visitarnos!
Bienvenido. Necesitas 